In [28]:
#Import necessary libraries
import cv2
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold

# Set up data and label paths
disease_path = 'C:/Users/samue/Downloads/Disease'

#Define HOG parameters
hog_orientations = 9
hog_pixels_per_cell = (8, 8)
hog_cells_per_block = (3, 3)
hog_block_norm = 'L2-Hys'

# Define input image shape
input_shape = (244, 244, 3)

# Initialize empty feature and label lists
features = []
labels = []

# Loop through each subdirectory and extract HOG features for each image
for label in ['PNEUMONIA', 'TURBERCULOSIS', 'COVID19']:
    label_path = os.path.join(disease_path, label)
    for image_file in os.listdir(label_path):
        # Load and resize image
        img = cv2.imread(os.path.join(label_path, image_file))
        img = cv2.resize(img, (input_shape[0], input_shape[1]))
        # Append the image and label to the lists
        features.append(img)
        labels.append(label)

# Convert features and labels to numpy arrays
features = np.array(features)
labels = np.array(labels)

# Encode labels as integers and convert to categorical
label_encoder = LabelEncoder()
labels = to_categorical(label_encoder.fit_transform(labels))

# Define CNN architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Initialize the StratifiedKFold cross validator
skf = KFold(n_splits=10, shuffle=True, random_state=42)

# Initialize empty list to store accuracy scores
accuracy_scores = []

# Loop through each fold and train the classifier
for fold_idx, (train_idx, test_idx) in enumerate(skf.split(features, labels)):
    # Split data into training and testing sets
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = labels[train_idx], labels[test_idx]
    # Train the classifier
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
    # Evaluate the classifier and store accuracy score
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    accuracy_scores.append(accuracy)
    # Print accuracy score for the current fold
    print(f"Fold {fold_idx+1}: Accuracy = {accuracy}")
    
# Print the average accuracy score across all folds
print(f"\nAverage accuracy = {np.mean(accuracy_scores)}")


Fold 1: Accuracy = 0.9555555582046509
Fold 2: Accuracy = 1.0
Fold 3: Accuracy = 1.0
Fold 4: Accuracy = 1.0
Fold 5: Accuracy = 1.0
Fold 6: Accuracy = 1.0
Fold 7: Accuracy = 1.0
Fold 8: Accuracy = 1.0
Fold 9: Accuracy = 1.0
Fold 10: Accuracy = 1.0

Average accuracy = 0.9955555558204651


In [29]:
# Evaluate the classifier on the entire dataset
y_pred_all = np.argmax(model.predict(features), axis=1)
y_true_all = np.argmax(labels, axis=1)
accuracy = metrics.accuracy_score(y_true_all, y_pred_all)
f1_score = metrics.f1_score(y_true_all, y_pred_all, average='weighted')
cohen_k_score = metrics.cohen_kappa_score(y_true_all, y_pred_all)

# Print the accuracy, f1, and cohen k scores
print("Accuracy:", accuracy)
print("F1 Score:", f1_score)
print("Cohen Kappa Score:", cohen_k_score)

# Calculate and print the confusion matrix for the classifier
confusion_matrix = metrics.confusion_matrix(y_true_all, y_pred_all)
print("Confusion Matrix:")
print(confusion_matrix)

# Get the confusion matrix
confusion_matrix = metrics.confusion_matrix(y_true_all, y_pred_all)

# Calculate the true positives, false positives, true negatives, and false negatives
TP = np.diag(confusion_matrix)
FP = confusion_matrix.sum(axis=0) - TP
FN = confusion_matrix.sum(axis=1) - TP
TN = confusion_matrix.sum() - (TP + FP + FN)

# Calculate the specificity and sensitivity
specificity = TN / (TN + FP)
sensitivity = TP / (TP + FN)

print("Specificity:", specificity)
print("Sensitivity:", sensitivity)

14/14 [==============================] - 3s 182ms/step
Accuracy: 1.0
F1 Score: 1.0
Cohen Kappa Score: 1.0
Confusion Matrix:
[[149   0   0]
 [  0 149   0]
 [  0   0 150]]
Specificity: [1. 1. 1.]
Sensitivity: [1. 1. 1.]
